In [2]:
import pulp
import pandas as pd
import numpy as np

In [3]:
# Define the problem
prob = pulp.LpProblem("Incoming_Bus_Allocation", pulp.LpMinimize)

In [4]:
# Define sets
solo_routes = ['ALABANG','BINAN','CARMONA','BALIBAGO','CABUYAO','CALAMBA']  # List of solo routes
hybrid_routes = [('ALABANG','CARMONA'),('BINAN','CARMONA'),('CALAMBA','CABUYAO')]  # List of hybrid routes (each element is a tuple (i, j))

# Import parameters (to be replaced with actual data import code)
cost_small_bus = {}  # Dictionary with costs for small buses on solo routes
cost_large_bus = {}  # Dictionary with costs for large buses on solo routes
cost_small_hybrid_route = {}  # Dictionary with costs for small hybrid routes
cost_large_hybrid_route = {}  # Dictionary with costs for large hybrid routes
capacity_small_bus = 18
capacity_large_bus = 56
buffer_current_small = {}  # Dictionary with buffer capacities for small buses
buffer_current_large = {}  # Dictionary with buffer capacities for large buses
demand = {}  # Dictionary of demands for each route

In [5]:
# Loading cost rates from the spreadsheet
rates_file_path = 'Bus Rate cleaned.xlsx'
xls = pd.ExcelFile(rates_file_path)

# Load the sheet into a DataFrame
cost_df = pd.read_excel(xls, sheet_name=0)

# Load the sheet into a DataFrame
cost_df = pd.read_excel(xls, sheet_name=0)

# Extracting costs
for index, row in cost_df.iterrows():
    route = row['ROUTE']
    total_large = row['TOTAL']
    total_small = row['TOTAL.1']
    
    # Check if it's a hybrid route
    if 'VIA' in route:
        route_parts = route.split(' VIA ')
        route_tuple = (route_parts[1], route_parts[0])
        cost_large_hybrid_route[route_tuple] = total_large
        cost_small_hybrid_route[route_tuple] = total_small
    else:
        cost_large_bus[route] = total_large
        cost_small_bus[route] = total_small

In [6]:
# Add bus capacities and buffer size

#Edit this buffer if needed. For now, it's a flat 3 people buffer per bus
buffer_small = 0
buffer_large = 3

for r in solo_routes:
    buffer_current_small[r]=buffer_small
    buffer_current_large[r]=buffer_large
    
for hr in hybrid_routes:
    buffer_current_small[hr]=buffer_small
    buffer_current_large[hr]=buffer_large

In [8]:
print(buffer_current_large)

{'ALABANG': 3, 'BINAN': 3, 'CARMONA': 3, 'BALIBAGO': 3, 'CABUYAO': 3, 'CALAMBA': 3, ('ALABANG', 'CARMONA'): 3, ('BINAN', 'CARMONA'): 3, ('CALAMBA', 'CABUYAO'): 3}


In [6]:
#Demands 

# Load the data
demand_file_path = 'cleaned_bus_shuttle_data_v31.csv'
bus_data = pd.read_csv(demand_file_path)

# Create a dictionary of dictionaries to store the 'ROUTE':'TOTAL' for each shift for each day
shift_data = {}

# Iterate through each unique date
for date in bus_data['DATE'].unique():
    shift_data[date] = {}
    date_data = bus_data[bus_data['DATE'] == date]
    
    # Iterate through each unique shift
    for shift in date_data['SHIFT'].unique():
        shift_data[date][shift] = {}
        shift_data_date_shift = date_data[date_data['SHIFT'] == shift]
        
        # Populate the dictionary for each route and its total
        for _, row in shift_data_date_shift.iterrows():
            route_name = row['ROUTE'].upper()
            shift_data[date][shift][route_name] = row['TOTAL']

# EDIT THIS IF NECESSARY
# Select one of the dictionaries for testing
test_date = '25'  # Select the date
test_shift = 'OUT 4PM'  # Select the shift
test_dict = shift_data[test_date][test_shift]

# Display the selected dictionary for testing
# test_dict

demand = test_dict

In [7]:
demand

{'ALABANG': 4,
 'BINAN': 6,
 'CARMONA': 4,
 'BALIBAGO': 31,
 'TAGAPO': 0,
 'CABUYAO': 4,
 'CALAMBA': 7}

In [8]:
# Preprocessing to exclude NaN values
def preprocess_costs(cost_dict):
    return {k: v for k, v in cost_dict.items() if not pd.isna(v)}

cost_small_bus = preprocess_costs(cost_small_bus)
cost_large_bus = preprocess_costs(cost_large_bus)
cost_small_hybrid_route = preprocess_costs(cost_small_hybrid_route)
cost_large_hybrid_route = preprocess_costs(cost_large_hybrid_route)

# Updated sets based on preprocessed costs
solo_routes = [route for route in solo_routes if route in cost_small_bus or route in cost_large_bus]
hybrid_routes = [route for route in hybrid_routes if route in cost_small_hybrid_route or route in cost_large_hybrid_route]

print("Preprocessed solo routes:", solo_routes)
print("Preprocessed hybrid routes:", hybrid_routes)
print("Preprocessed cost_small_bus:", cost_small_bus)
print("Preprocessed cost_large_bus:", cost_large_bus)
print("Preprocessed cost_small_hybrid_route:", cost_small_hybrid_route)
print("Preprocessed cost_large_hybrid_route:", cost_large_hybrid_route)

Preprocessed solo routes: ['ALABANG', 'BINAN', 'CARMONA', 'BALIBAGO', 'CABUYAO', 'CALAMBA']
Preprocessed hybrid routes: [('ALABANG', 'CARMONA'), ('BINAN', 'CARMONA'), ('CALAMBA', 'CABUYAO')]
Preprocessed cost_small_bus: {'ALABANG': 1423.5, 'BINAN': 586.5, 'BALIBAGO': 1038.0, 'CABUYAO': 586.5, 'CALAMBA': 1300.5}
Preprocessed cost_large_bus: {'ALABANG': 3072, 'BINAN': 2460, 'CARMONA': 1881, 'BALIBAGO': 1881, 'CABUYAO': 1935, 'CALAMBA': 2127}
Preprocessed cost_small_hybrid_route: {('ALABANG', 'CARMONA'): 1518.0, ('BINAN', 'CARMONA'): 630.0, ('CALAMBA', 'CABUYAO'): 630.0}
Preprocessed cost_large_hybrid_route: {('ALABANG', 'CARMONA'): 3270, ('BINAN', 'CARMONA'): 2550, ('CALAMBA', 'CABUYAO'): 2625}


In [9]:
# Decision Variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Objective Function
prob += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if cost_small_bus.get(i, 0) > 0]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if cost_large_bus.get(i, 0) > 0]) +
    pulp.lpSum([cost_small_hybrid_route.get((i, j), 0) * y_small[(i, j)] for (i, j) in hybrid_routes if cost_small_hybrid_route.get((i, j), 0) > 0]) +
    pulp.lpSum([cost_large_hybrid_route.get((i, j), 0) * y_large[(i, j)] for (i, j) in hybrid_routes if cost_large_hybrid_route.get((i, j), 0) > 0])
)

# Constraints
for i in solo_routes:
    prob += (
        (capacity_small_bus * x_small[i] if cost_small_bus.get(i, 0) > 0 else 0) +
        (capacity_large_bus * x_large[i] if cost_large_bus.get(i, 0) > 0 else 0) +
        pulp.lpSum([capacity_small_bus * y_small[(i, j)] for j in solo_routes if cost_small_hybrid_route.get((i, j)) is not None]) +
        pulp.lpSum([capacity_large_bus * y_large[(i, j)] for j in solo_routes if cost_large_hybrid_route.get((i, j)) is not None]) +
        pulp.lpSum([capacity_small_bus * y_small[(j, i)] for j in solo_routes if cost_small_hybrid_route.get((j, i)) is not None]) +
        pulp.lpSum([capacity_large_bus * y_large[(j, i)] for j in solo_routes if cost_large_hybrid_route.get((j, i)) is not None])
        >= demand.get(i, 0), f"Demand_Constraint_{i}"
    )

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total Cost = ", pulp.value(prob.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/hk/l0w5001d7pdc860p4ssywh3h0000gn/T/eee1eab5dc45477794f8c7b7dc1c60a1-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hk/l0w5001d7pdc860p4ssywh3h0000gn/T/eee1eab5dc45477794f8c7b7dc1c60a1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 86 RHS
At line 93 BOUNDS
At line 111 ENDATA
Problem MODEL has 6 rows, 17 columns and 23 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1710.86 - 0.00 seconds
Cgl0003I 0 fixed, 17 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 

In [10]:
for name, constraint in prob.constraints.items():
    print(f"Constraint {name}: {constraint}")

Constraint Demand_Constraint_ALABANG: 56*x_large_ALABANG + 18*x_small_ALABANG + 56*y_large_('ALABANG',_'CARMONA') + 18*y_small_('ALABANG',_'CARMONA') >= 4
Constraint Demand_Constraint_BINAN: 56*x_large_BINAN + 18*x_small_BINAN + 56*y_large_('BINAN',_'CARMONA') + 18*y_small_('BINAN',_'CARMONA') >= 6
Constraint Demand_Constraint_CARMONA: 56*x_large_CARMONA + 56*y_large_('ALABANG',_'CARMONA') + 56*y_large_('BINAN',_'CARMONA') + 18*y_small_('ALABANG',_'CARMONA') + 18*y_small_('BINAN',_'CARMONA') >= 4
Constraint Demand_Constraint_BALIBAGO: 56*x_large_BALIBAGO + 18*x_small_BALIBAGO >= 31
Constraint Demand_Constraint_CABUYAO: 56*x_large_CABUYAO + 18*x_small_CABUYAO + 56*y_large_('CALAMBA',_'CABUYAO') + 18*y_small_('CALAMBA',_'CABUYAO') >= 4
Constraint Demand_Constraint_CALAMBA: 56*x_large_CALAMBA + 18*x_small_CALAMBA + 56*y_large_('CALAMBA',_'CABUYAO') + 18*y_small_('CALAMBA',_'CABUYAO') >= 7
